<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/gpts/xplainT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-05 08:29:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.00MB/s    in 0.3s    

2025-07-05 08:29:29 (4.00 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open(mode="r", file="input.txt") as f:
  text = f.read()

text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [4]:
vocab = sorted(set(list(text)))

In [5]:
stoi = {v : k for k, v in enumerate(vocab)}
itos = {k : v for k, v in enumerate(vocab)}

encode = lambda seq : [stoi[s] for s in seq]
decode = lambda idx : "".join([itos[x] for x in idx])

In [6]:
data = torch.tensor(encode(text))

In [7]:
###################
# HYPERPARAMETERS #
###################
embed_dim = 32
block_size = 8

In [43]:
tokenized_input = torch.randint(len(vocab), (1, block_size))
tokenized_input

tensor([[32, 59, 30, 64, 47,  6, 21,  4]])

In [44]:
token_embedding = nn.Embedding(len(vocab), embed_dim)

In [45]:
tok_embd = token_embedding(tokenized_input)
tok_embd

tensor([[[ 4.3306e-01,  4.2382e-01,  1.1437e+00,  3.2522e+00, -4.7278e-01,
          -7.7109e-01,  1.7602e+00,  7.9652e-01, -2.4368e-01, -7.3788e-01,
           3.5720e-01,  9.4277e-01, -2.2218e-01,  1.1993e+00,  1.0146e+00,
           1.8471e-01,  4.2926e-01, -5.0482e-01,  1.8994e-01,  1.4611e+00,
           4.8480e-02,  1.1599e+00, -2.9260e+00,  5.8502e-01,  3.6411e-01,
          -1.7409e+00, -3.6501e-01,  7.7247e-01,  3.0108e-01, -2.9392e-01,
          -1.4681e+00, -5.1487e-01],
         [ 4.9305e-01, -2.0926e+00,  3.1621e-02,  1.5678e+00,  8.0986e-01,
           1.0023e+00, -8.5893e-01, -4.2041e-01,  2.0969e-02,  9.8109e-03,
          -2.7307e-01,  1.7060e-01, -1.7654e+00,  1.8980e-02, -4.6169e-01,
           1.7188e+00, -1.1971e+00, -7.2910e-01, -4.2490e-01, -5.3011e-01,
           1.0966e+00, -8.0194e-01, -8.8083e-02, -1.3399e+00, -6.9621e-01,
           1.8115e-01, -1.2060e+00, -9.3327e-01,  9.0301e-01,  9.0591e-01,
          -2.7146e-01,  9.0703e-01],
         [ 1.9537e-01, -1.

In [48]:
positional_embedding = nn.Embedding(block_size, embed_dim)

In [49]:
pos_embd = positional_embedding(torch.arange(block_size))
pos_embd

tensor([[ 2.7583e-01, -4.6276e-01,  1.0298e+00,  7.3756e-01, -5.4005e-01,
         -5.0844e-01, -3.0195e-01, -1.9385e-01, -3.5528e-01,  4.7202e-01,
         -2.5018e+00, -2.6457e-02,  1.9488e-01, -1.3831e+00,  1.1748e+00,
          3.8228e-01, -2.6083e-01, -5.3551e-01,  2.0576e-01, -1.1596e+00,
          3.4783e-01,  8.7152e-02, -1.6595e+00,  1.5351e+00, -2.1600e-01,
          1.3247e+00, -3.8743e-01, -1.2809e+00, -1.7851e+00,  7.7544e-01,
          1.4290e+00,  2.8740e-01],
        [-2.1464e+00, -8.0020e-01,  2.1941e+00,  2.5748e-01,  4.3982e-01,
          2.7914e-01, -1.2735e+00,  8.3904e-01, -1.5515e+00, -8.3550e-02,
          2.2908e-02, -1.4393e+00,  3.3243e-01, -9.1067e-01,  1.9752e-01,
         -1.5836e+00,  7.0234e-01, -2.6374e+00, -7.7605e-01,  8.7082e-01,
          5.9474e-01, -5.2444e-01,  2.9371e-01,  1.4642e+00,  1.2392e+00,
         -9.5420e-01, -2.1819e-01, -1.4206e-01,  7.2939e-01, -8.5025e-01,
          2.5472e-01, -1.2288e+00],
        [-1.4772e+00,  5.3288e-01,  8.45

In [50]:
tok_pos_embd = tok_embd + pos_embd
tok_pos_embd.shape

torch.Size([1, 8, 32])

In [61]:
query = nn.Linear(embed_dim, embed_dim)
key = nn.Linear(embed_dim, embed_dim)
value = nn.Linear(embed_dim, embed_dim)
tril = torch.tril(torch.ones(block_size, block_size))

In [62]:
q = query(tok_embd)
k = key(tok_embd)
v = value(tok_embd)

In [65]:
wei = q @ k.transpose(-2, -1)
wei = wei.masked_fill(tril[:block_size, :block_size] == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([1, 8, 32])